## Cross-Bucket Tests on all Acc/Dec and Random Fragments Kalman Filter

### Part A: (1) Import Fragment Data from MATLAB, (2) Save Them in a .pickle file, and (3) Load Them In

###### (HOWEVER, YOU DON'T NEED TO LOAD THEM IN AGAIN SINCE STEP 1 ALREADY DOES THAT FOR YOU!)

In [119]:
## (Configuration) Allows you to return multiple variables from a single cell ##
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Allows you to import files from another folder in current directory ## 
import os 
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [120]:
#Import standard packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import io
from scipy import stats
import pickle
import sys

#Import metrics
from Neural_Decoding.metrics import get_R2
from Neural_Decoding.metrics import get_rho
from Neural_Decoding.metrics import get_R2_parts

#Import decoder functions
from Neural_Decoding.decoders import KalmanFilterDecoder

### Part B: Preprocessing Decoder Data

In [246]:
## Import Data ##
# We'll load in position data and derive velocity and acceleration from it 

# Specify Fragment Types to be used for this anaylsis 
frag_type = ['AD', 'Rand'] 
# Specify folder where MATLAB data is stored
folder = '/Users/rbhatt6/Documents/MATLAB/' 

 #locals()["sortIn"+frag_type[i]] = io.loadmat(folder+'cleanedSortIn.mat')
for i in range(len(frag_type)):
    input = "sortIn_"+frag_type[i]
    locals()[input] = io.loadmat(folder + input + '.mat')

    output = "sortOut_" + frag_type[i]
    locals()[output] = io.loadmat(folder + output + '.mat')

    locals()[input] = np.squeeze(list(locals()[input].values())[3])
    locals()[output] = np.squeeze(list(locals()[output].values())[3])

In [247]:
## Format Kinematics Data (outputs) ##
# For the Kalman filter, we use the position, velocity, and acceleration as outputs.
# Ultimately, we are only concerned with the goodness of fit of velocity, but using them all as covariates helps performance.

for i in range(len(frag_type)):
    # Pulling local variables into new, temp variables
    output = locals()["sortOut_"+frag_type[i]]
    
    # Creating a local variable for each decoder_output
    decoder_output = "decoder_output_" + frag_type[i]
    locals()[decoder_output] = []

    for j in range(len(output)): # Number of buckets (i.e. 16 or 8)
        nFrags = output[j][0].shape[0]
        temp2 = []
        for k in range(0, nFrags, 1): # Number of fragments #output[0][0].shape[0]
            vel_X = float(output[j][0][k])
            vel_Y = float(output[j][1][k])
            acc_X = float(output[j][2][k])
            acc_Y = float(output[j][3][k])
            pos_X = float(output[j][4][k])
            pos_Y = float(output[j][5][k])
            temp = [vel_X, vel_Y, acc_X, acc_Y, pos_X, pos_Y]
            temp2.append(np.array(temp))
            #locals()[decoder_output][j][k].append(np.array(temp))
        #locals()[decoder_output][j].append(np.array(temp2))
        locals()[decoder_output].append(np.array(temp2))
        #temp = np.array(np.concatenate((vel_X, vel_Y, acc_X, acc_Y, outputX[j], outputY[j]),axis=1))

### Part C: Partitioning and Running the Kalman Filter on the Same Buckets 

In [ ]:
# (NEW) CROSS-VALIDATED WITHIN-BUCKET TEST (for all 16 buckets)
# Doing a 10-fold cross validation procedure

# for loop through 10 folds then 16 buckets

for i in range(len(frag_type)):

   # Pulling local variables 
   input = locals()["sortIn_"+frag_type[i]]
   output = locals()["decoder_output_"+frag_type[i]]

   # Creating a local variable to hold final training and testing data 
   final_X_train = "final_X_train_" + frag_type[i]
   final_y_train = "final_y_train_" + frag_type[i]
   final_X_test = "final_X_test_" + frag_type[i]
   final_y_test = "final_y_test_" + frag_type[i]
   locals()[final_X_train] = []
   locals()[final_y_train] = []
   locals()[final_X_test] = []
   locals()[final_y_test] = []

   idx_list = [0,1,2,3,4,5,6,7,8,9]

   # Splitting data for a single bucket into 10 folds - (j) is the left-out fold for testing
   for j in range(0,10):
      test_idx = j
      train_idx = idx_list.copy()
      train_idx.pop(j)

      temp_X_train = []
      temp_y_train = []
      temp_X_test = []
      temp_y_test = []
      
      for k in range(len(input)):
         X_split = np.array_split(input[k], 10)
         y_split = np.array_split(output[k], 10)

         # Index the list of indices to create a list of np ndarrays for training (9 arrays) and testing (1 array)
         X_train = [X_split[i] for i in train_idx]
         X_test = X_split[j]
         y_train = [y_split[i] for i in train_idx]
         y_test = y_split[j]

         # Concatenate the list of 9 arrays for the training sets 
         X_train = np.concatenate(X_train, axis=0)
         y_train = np.concatenate(y_train, axis=0)
         
         temp_X_train.append(X_train)
         temp_y_train.append(y_train)
         temp_X_test.append(X_test)
         temp_y_test.append(y_test)

      locals()[final_X_train].append(temp_X_train)
      locals()[final_y_train].append(temp_y_train)
      locals()[final_X_test].append(temp_X_test)
      locals()[final_y_test].append(temp_y_test)

## Check for Decoder Accuracy
### Training on 15 buckets and Testing on 1 bucket instead

In [176]:
# Within-Bucket Test with 10-fold Cross-Validation 
from Neural_Decoding.runModelsKF import run_model_kf_test

for i in range(len(frag_type)):

    # Pulling local variables 
    frag_X_train =  locals()["final_X_train_" + frag_type[i]]
    frag_y_train = locals()["final_y_train_" + frag_type[i]]
    frag_X_test = locals()["final_X_test_" + frag_type[i]]
    frag_y_test = locals()["final_y_test_" + frag_type[i]]

    # Creating a local variable to hold predicted outputs for each frag type AND their trained models
    parts = "pred_parts_" + frag_type[i]
    models = "more_trained_models_" + frag_type[i]
    locals()[parts] = []
    locals()[models] = []

    for cross in range(0,10):
        X_train = frag_X_train[cross]
        y_train = frag_y_train[cross]
        X_test = frag_X_test[cross]
        y_test = frag_y_test[cross]
        
        curr_R2s, curr_models = run_model_kf_test(X_train, y_train, X_test,y_test, "parts")
        locals()[parts].append(curr_R2s)
        locals()[models].append(curr_models)

In [177]:
# Compute Within-Bucket combined XY_FVAF 
# For velocity only, although all kinematic variables should be used for model-fitting and predicted for.
from Neural_Decoding.metrics import compute_XY_FVAF

# 'pred_parts_AD' was thankfully also a list of arrays containing the R2s

for i in range(len(frag_type)):

    # Pulling local variables (computed R2s for parts)
    R2s_parts = locals()["pred_parts_" + frag_type[i]]

    # Creating a local variable to hold final XY_FVAFs
    XY_FVAFs = "XY_FVAFs_" + frag_type[i]
    locals()[XY_FVAFs] = []

    for j in range(0,10):
        curr_fold = R2s_parts[j]

        curr_fold_XY_FVAFs = []
        for k in range(0,16):
            #curr_bucket = Kalman_R2s_combined[i]
            vel_x_nom = curr_fold[k][0][0] # dim = (curr_bucket, nom, x_vel)
            vel_x_denom = curr_fold[k][1][0] # dim = (curr_bucket, denom, x_vel)
            vel_y_nom = curr_fold[k][0][1] # dim = (curr_bucket, nom, y_vel)
            vel_y_denom = curr_fold[k][1][1] # dim = (curr_bucket, denom, y_vel)

            XY_FVAF = compute_XY_FVAF(vel_x_nom,vel_x_denom,vel_y_nom,vel_y_denom)
            curr_fold_XY_FVAFs.append(XY_FVAF)

        locals()[XY_FVAFs].append(curr_fold_XY_FVAFs)

In [179]:
XY_FVAFs_AD

[[-1.4571230620931397,
  -1.1846134048645154,
  -0.8207205862457452,
  -0.814745116907047,
  -1.292928409702053,
  -0.598931898853164,
  -0.5927502258960964,
  -0.834487843214043,
  -1.502806437404249,
  -1.1421879968808892,
  -0.6072785659945226,
  -1.6211931176767784,
  -1.5300251076664293,
  -1.4794785913879176,
  -0.39013945411981954,
  -0.8857477211320386],
 [-2.840569765698738,
  -1.3167112352173604,
  -0.6732968194618283,
  -0.927594528375902,
  -0.9313015474525199,
  -1.113326374106212,
  -0.4386858201102286,
  -0.8898030129232035,
  -1.4970654531485859,
  -0.9112670683195472,
  -0.6500987693683551,
  -0.9076036697229148,
  -1.321889866728172,
  -1.1334468033302576,
  -0.7496156981400901,
  -1.3791216005316809],
 [-1.5317783045048774,
  -1.172033436794972,
  -0.5003569383156901,
  -0.9404624349808788,
  -1.6798137482270588,
  -0.80311275561573,
  -0.6262415431837096,
  -1.0223720033166095,
  -1.469289952539524,
  -1.0029809889545436,
  -0.4777713461757569,
  -1.0425264466772841

In [76]:
# Saving results in a matlab file 
from scipy.io import savemat

result1 = [XY_FVAFs_AD, XY_FVAFs_Rand]
FrameStack1 = np.empty((2,), dtype=object)
for i in range(len(result1)):
    FrameStack1[i] = result1[i]

#savemat("Acoss_All_Buckets_100neurons.mat", {"Across_All_Buckets_100_neurons":FrameStack1})
savemat("Acoss_All_Buckets_89neurons.mat", {"Across_All_Buckets_89_neurons":FrameStack1})

## Training and Testing Models for Same Direction Or Same Polarity Only 

In [249]:
# Direction-Specific Test
from Neural_Decoding.runModelsKF import run_model_kf_direction

for i in range(len(frag_type)):

    # Pulling local variables 
    frag_X_train =  locals()["final_X_train_" + frag_type[i]]
    frag_y_train = locals()["final_y_train_" + frag_type[i]]
    frag_X_test = locals()["final_X_test_" + frag_type[i]]
    frag_y_test = locals()["final_y_test_" + frag_type[i]]

    # Creating a local variable to hold predicted outputs for each frag type AND their trained models
    parts = "pred_parts_" + frag_type[i]
    #models = "direction_specific_models_" + frag_type[i]
    locals()[parts] = []
    #locals()[models] = []

    for cross in range(0,10):
        X_train = frag_X_train[cross]
        y_train = frag_y_train[cross]
        X_test = frag_X_test[cross]
        y_test = frag_y_test[cross]
        
        curr_R2s, curr_models = run_model_kf_direction(X_train, y_train, X_test,y_test, "parts")
        locals()[parts].append(curr_R2s)
        #locals()[models].append(curr_models)

In [250]:
from Neural_Decoding.metrics import compute_XY_FVAF

for i in range(len(frag_type)):

    # Pulling local variables (computed R2s for parts)
    R2s_parts = locals()["pred_parts_" + frag_type[i]]

    # Creating a local variable to hold final XY_FVAFs
    XY_FVAFs = "direction_specific_XY_FVAFs_" + frag_type[i]
    locals()[XY_FVAFs] = []

    for j in range(0,10):
        curr_fold = R2s_parts[j]

        curr_fold_XY_FVAFs = []
        for k in range(0,8):
            #curr_bucket = Kalman_R2s_combined[i]
            vel_x_nom = curr_fold[k][0][0] # dim = (curr_bucket, nom, x_vel)
            vel_x_denom = curr_fold[k][1][0] # dim = (curr_bucket, denom, x_vel)
            vel_y_nom = curr_fold[k][0][1] # dim = (curr_bucket, nom, y_vel)
            vel_y_denom = curr_fold[k][1][1] # dim = (curr_bucket, denom, y_vel)

            XY_FVAF = compute_XY_FVAF(vel_x_nom,vel_x_denom,vel_y_nom,vel_y_denom)
            curr_fold_XY_FVAFs.append(XY_FVAF)

        locals()[XY_FVAFs].append(curr_fold_XY_FVAFs)

In [251]:
direction_specific_XY_FVAFs_Rand

[[0.21695834845165074,
  0.09371795094109614,
  0.1595158364193231,
  0.17227110237801224,
  0.10432952212691482,
  0.08254187381287825,
  0.20121796818156856,
  0.12821236588067386],
 [0.21197082686873225,
  0.18204884935510546,
  0.15526313578604434,
  0.2081088678088292,
  0.16825448225980955,
  0.0416625651442144,
  0.1358824459805137,
  0.1873656836627],
 [0.12718850455387376,
  0.08789548362589195,
  0.13981736118568855,
  0.15133551650771015,
  0.13739949590080003,
  0.13971239311451888,
  0.17916572506215833,
  0.1280446694911267],
 [0.1672258426423291,
  0.13462951614332808,
  0.20758468331779167,
  0.034945980578425795,
  0.07663979887104888,
  0.09125737684755142,
  0.11301868465568798,
  0.16545115987942927],
 [0.21728167770054796,
  0.20620866523844794,
  0.163943287310377,
  0.08577363463589849,
  0.1155274949766173,
  0.1536971425780448,
  0.16624825752540606,
  0.11977142432401111],
 [0.0970237655253372,
  0.1685561730062689,
  0.1846543902299972,
  0.1354225654897454,


In [252]:
# Saving results in a matlab file 
from scipy.io import savemat

result1 = [direction_specific_XY_FVAFs_AD, direction_specific_XY_FVAFs_Rand]

FrameStack1 = np.empty((2,), dtype=object)

for i in range(len(result1)):
    FrameStack1[i] = result1[i]
savemat("Direction_Specific_Buckets_3.mat", {"Direction_Specific_XY_FVAFs_3":FrameStack1})

In [254]:
# Polarity-Specific Test
from Neural_Decoding.runModelsKF import run_model_kf_polarity

for i in range(len(frag_type)):

    # Pulling local variables 
    frag_X_train =  locals()["final_X_train_" + frag_type[i]]
    frag_y_train = locals()["final_y_train_" + frag_type[i]]
    frag_X_test = locals()["final_X_test_" + frag_type[i]]
    frag_y_test = locals()["final_y_test_" + frag_type[i]]

    # Creating a local variable to hold predicted outputs for each frag type AND their trained models
    parts = "pred_parts_" + frag_type[i]
    models = "polarity_specific_models_" + frag_type[i]
    locals()[parts] = []
    locals()[models] = []

    for cross in range(0,10):
        X_train = frag_X_train[cross]
        y_train = frag_y_train[cross]
        X_test = frag_X_test[cross]
        y_test = frag_y_test[cross]
        
        curr_R2s, curr_models = run_model_kf_polarity(X_train, y_train, X_test,y_test, "parts")
        locals()[parts].append(curr_R2s)
        locals()[models].append(curr_models)

In [255]:
from Neural_Decoding.metrics import compute_XY_FVAF

for i in range(len(frag_type)):

    # Pulling local variables (computed R2s for parts)
    R2s_parts = locals()["pred_parts_" + frag_type[i]]

    # Creating a local variable to hold final XY_FVAFs
    XY_FVAFs = "polarity_specific_XY_FVAFs_" + frag_type[i]
    locals()[XY_FVAFs] = []

    for j in range(0,10):
        curr_fold = R2s_parts[j]

        curr_fold_XY_FVAFs = []
        for k in range(0,2):
            #curr_bucket = Kalman_R2s_combined[i]
            vel_x_nom = curr_fold[k][0][0] # dim = (curr_bucket, nom, x_vel)
            vel_x_denom = curr_fold[k][1][0] # dim = (curr_bucket, denom, x_vel)
            vel_y_nom = curr_fold[k][0][1] # dim = (curr_bucket, nom, y_vel)
            vel_y_denom = curr_fold[k][1][1] # dim = (curr_bucket, denom, y_vel)

            XY_FVAF = compute_XY_FVAF(vel_x_nom,vel_x_denom,vel_y_nom,vel_y_denom)
            curr_fold_XY_FVAFs.append(XY_FVAF)

        locals()[XY_FVAFs].append(curr_fold_XY_FVAFs)

In [258]:
polarity_specific_XY_FVAFs_AD

[[0.3386737179256073, 0.25980955960403607],
 [0.3280304777878995, 0.218101291696028],
 [0.31791993941680063, 0.279729433204163],
 [0.2705727645254551, 0.23530381958004198],
 [0.2972852429907651, 0.23191511994156255],
 [0.35858105911466065, 0.2519671485653425],
 [0.2571985780195857, 0.1818187942326448],
 [0.3704386117069062, 0.27776864433615767],
 [0.3205895418018442, 0.22246253351950163],
 [0.3349608508140536, 0.23074439990865803]]

In [260]:
# Saving results in a matlab file 
from scipy.io import savemat

# result1 = [polarity_specific_XY_FVAFs_AD, polarity_specific_test_XY_FVAFs]
result1 = [polarity_specific_XY_FVAFs_AD, polarity_specific_XY_FVAFs_Rand]
FrameStack1 = np.empty((2,), dtype=object)

for i in range(len(result1)):
    FrameStack1[i] = result1[i]

savemat("Polarity_Specific_Buckets_Adjusted.mat", {"Polarity_Specific_XY_FVAFs_Adjusted":FrameStack1})

In [175]:
# Trained Models for Specific Polarities are Now Separately Tested
#(i.e. testing models trained on Random Fragments on AD fragments)
from Neural_Decoding.runModelsKF import run_model_kf_polarity_test

# Creating a local variable to hold predicted outputs for each frag type AND their trained models
polarity_specific_test_XY_FVAFs = []

for cross in range(0,10):
    curr_models = polarity_specific_models_Rand[cross]
    X_test = final_X_test_AD[cross]
    y_test = final_y_test_AD[cross]
    polarity_specific_test_XY_FVAFs.append(run_model_kf_polarity_test(curr_models, X_test, y_test))
    

ValueError: operands could not be broadcast together with shapes (85,1) (80,1) 

In [148]:
polarity_specific_test_XY_FVAFs

[]

## Training and Testing Models on Same Bucket

In [ ]:
# (NEW) Within-Bucket Test with 10-fold Cross-Validation 
from Neural_Decoding.runModelsKF import run_model_kf_cv

for i in range(len(frag_type)):

    # Pulling local variables 
    frag_X_train =  locals()["final_X_train_" + frag_type[i]]
    frag_y_train = locals()["final_y_train_" + frag_type[i]]
    frag_X_test = locals()["final_X_test_" + frag_type[i]]
    frag_y_test = locals()["final_y_test_" + frag_type[i]]

    # Creating a local variable to hold predicted outputs for each frag type AND their trained models
    parts = "pred_parts_" + frag_type[i]
    models = "models_" + frag_type[i]
    locals()[parts] = []
    locals()[models] = []

    for cross in range(0,10):
        X_train = frag_X_train[cross]
        y_train = frag_y_train[cross]
        X_test = frag_X_test[cross]
        y_test = frag_y_test[cross]
        
        curr_R2s, curr_models = run_model_kf_cv(X_train, y_train, X_test,y_test, "parts")
        locals()[parts].append(curr_R2s)
        locals()[models].append(curr_models)


# len(within_bucket_R2s) # 10
# len(within_bucket_R2s[0]) # 16
# within_bucket_R2s[0][0] # each fold has R2s (vel,pos,acc) for each of the 16 buckets

In [110]:
# (NEW) Compute Within-Bucket combined XY_FVAF 
# For velocity only, although all kinematic variables should be used for model-fitting and predicted for.
from Neural_Decoding.metrics import compute_XY_FVAF

# 'pred_parts_AD' was thankfully also a list of arrays containing the R2s

for i in range(len(frag_type)):

    # Pulling local variables (computed R2s for parts)
    R2s_parts = locals()["pred_parts_" + frag_type[i]]

    # Creating a local variable to hold final XY_FVAFs
    XY_FVAFs = "XY_FVAFs_" + frag_type[i]
    locals()[XY_FVAFs] = []

    for j in range(0,10):
        curr_fold = R2s_parts[j]

        curr_fold_XY_FVAFs = []
        for k in range(0,16):
            #curr_bucket = Kalman_R2s_combined[i]
            vel_x_nom = curr_fold[k][0][0] # dim = (curr_bucket, nom, x_vel)
            vel_x_denom = curr_fold[k][1][0] # dim = (curr_bucket, denom, x_vel)
            vel_y_nom = curr_fold[k][0][1] # dim = (curr_bucket, nom, y_vel)
            vel_y_denom = curr_fold[k][1][1] # dim = (curr_bucket, denom, y_vel)

            XY_FVAF = compute_XY_FVAF(vel_x_nom,vel_x_denom,vel_y_nom,vel_y_denom)
            curr_fold_XY_FVAFs.append(XY_FVAF)

        locals()[XY_FVAFs].append(curr_fold_XY_FVAFs)

In [ ]:
len(XY_FVAFs_Rand)
len(XY_FVAFs_Rand[0])

XY_FVAFs_AD

In [113]:
# Saving results in a matlab file 
from scipy.io import savemat

result1 = [XY_FVAFs_AD, XY_FVAFs_Rand]

FrameStack1 = np.empty((2,), dtype=object)

for i in range(len(result1)):
    FrameStack1[i] = result1[i]

savemat("Within_Bucket.mat", {"Within_Bucket_XY_FVAFs":FrameStack1})

## Cross-Bucket Tests

In [115]:
# (NEW) Opposite-polarity, Opposite-direction, Complete-opposite and Full cross-bucket tests
from Neural_Decoding.runModelsKF import opposite_polarity_test_cv, complete_opposite_bucket_test_cv, opposite_direction_test_cv, cross_buckets_test_cv

for i in range(len(frag_type)):

    # Pulling local variables 
    frag_X_test = locals()["final_X_test_" + frag_type[i]]
    frag_y_test = locals()["final_y_test_" + frag_type[i]]
    models = locals()["models_" + frag_type[i]]

    # Creating a local variable to hold FVAFs
    opposite_polarity = "opposite_polarity_FVAFs_" + frag_type[i]
    opposite_direction = "opposite_direction_FVAFs_" + frag_type[i]
    complete_opposite = "complete_opposite_FVAFs_" + frag_type[i]
    #cross_buckets = "cross_buckets_FVAFs_" + frag_type[i]
    locals()[opposite_polarity] = []
    locals()[opposite_direction] = []
    locals()[complete_opposite] = []
    #locals()[cross_buckets] = []


    for j in range(0,10):
        curr_models = models[j]
        curr_X_test = frag_X_test[j]
        curr_y_test = frag_y_test[j]

        # Creating a local variable to hold opposite-polarity and complete-opposite FVAFs
        locals()[opposite_polarity].append(opposite_polarity_test_cv(curr_models, curr_X_test, curr_y_test, "parts", frag_type[i]))
        locals()[opposite_direction].append(opposite_direction_test_cv(curr_models, curr_X_test, curr_y_test, "parts", frag_type[i]))
        locals()[complete_opposite].append(complete_opposite_bucket_test_cv(curr_models, curr_X_test, curr_y_test, "parts", frag_type[i]))
        #locals()[cross_buckets].append(cross_buckets_test_cv(curr_models, curr_X_test, curr_y_test, "parts"))
        


In [ ]:
opposite_polarity_FVAFs_AD
opposite_direction_FVAFs_AD

In [118]:
# Saving results in a matlab file 
from scipy.io import savemat

result1 = [opposite_polarity_FVAFs_AD, opposite_polarity_FVAFs_Rand]
result2 = [opposite_direction_FVAFs_AD, opposite_direction_FVAFs_Rand]
result3 = [complete_opposite_FVAFs_AD, complete_opposite_FVAFs_Rand]

FrameStack1 = np.empty((2,), dtype=object)
FrameStack2 = np.empty((2,), dtype=object)
FrameStack3 = np.empty((2,), dtype=object)
for i in range(len(result1)):
    FrameStack1[i] = result1[i]
    FrameStack2[i] = result2[i]
    FrameStack3[i] = result3[i]
savemat("Opposite_Polarity.mat", {"Opposite_Polarity":FrameStack1})
savemat("Opposite_Direction.mat", {"Opposite_Direction":FrameStack2})
savemat("Complete_Opposite.mat", {"Complete_Opposite":FrameStack3})

### Part D: Partitioning and Running the Kalman Filter on Separate Training and Test Buckets

### Part E: Save all outputs (i.e. FVAFs)

In [23]:
# Saving results in a matlab file 
from scipy.io import savemat

result1 = [XY_FVAFs_AD, XY_FVAFs_Rand]
result2 = [cross_buckets_FVAFs_AD, cross_buckets_FVAFs_Rand]
result3 = [opposite_polarity_FVAFs_AD, opposite_polarity_FVAFs_Rand]
result4 = [opposite_direction_FVAFs_AD, opposite_direction_FVAFs_Rand]
result5 = [complete_opposite_FVAFs_AD, complete_opposite_FVAFs_Rand]

FrameStack1 = np.empty((2,), dtype=object)
FrameStack2 = np.empty((2,), dtype=object)
FrameStack3 = np.empty((2,), dtype=object)
FrameStack4 = np.empty((2,), dtype=object)
FrameStack5 = np.empty((2,), dtype=object)
for i in range(len(result1)):
    FrameStack1[i] = result1[i]
    FrameStack2[i] = result2[i]
    FrameStack3[i] = result3[i]
    FrameStack4[i] = result4[i]
    FrameStack5[i] = result5[i]
savemat("Within_Bucket.mat", {"XY_FVAF":FrameStack1})
savemat("Cross_Bucket.mat", {"Cross_Bucket":FrameStack2})
savemat("Opposite_Polarity.mat", {"Opposite_Polarity":FrameStack3})
savemat("Opposite_Direction.mat", {"Opposite_Direction":FrameStack4})
savemat("Complete_Opposite.mat", {"Complete_Opposite":FrameStack5})

### Notes

In [ ]:
for i in range(len(sortIn_AD)):
    X_kf = sortIn_AD[i]
    y_kf = decoder_output_AD[i]
    num_examples_kf=X_kf.shape[0] # nRows (b/c nCols = number of units)
    
    #Note that each range has a buffer of 1 bin at the beginning and end
    #This makes it so that the different sets don't include overlapping data
    training_set=np.arange(int(np.round(training_range[0]*num_examples_kf))+1,int(np.round(training_range[1]*num_examples_kf))-1)
    testing_set=np.arange(int(np.round(testing_range[0]*num_examples_kf))+1,int(np.round(testing_range[1]*num_examples_kf))-1)
    valid_set=np.arange(int(np.round(valid_range[0]*num_examples_kf))+1,int(np.round(valid_range[1]*num_examples_kf))-1)

    #Get training data
    X_kf_train=X_kf[training_set,:]
    y_kf_train=y_kf[training_set,:]

    #Get testing data
    X_kf_test=X_kf[testing_set,:]
    y_kf_test=y_kf[testing_set,:]

    #Get validation data
    X_kf_valid=X_kf[valid_set,:]
    y_kf_valid=y_kf[valid_set,:]

    #Z-score inputs 
    X_kf_train_mean=np.nanmean(X_kf_train,axis=0)
    X_kf_train_std=np.nanstd(X_kf_train,axis=0)
    X_kf_train=(X_kf_train-X_kf_train_mean)/X_kf_train_std
    X_kf_test=(X_kf_test-X_kf_train_mean)/X_kf_train_std
    X_kf_valid=(X_kf_valid-X_kf_train_mean)/X_kf_train_std

    #Zero-center outputs
    y_kf_train_mean=np.mean(y_kf_train,axis=0)
    y_kf_train=y_kf_train-y_kf_train_mean
    y_kf_test=y_kf_test-y_kf_train_mean
    y_kf_valid=y_kf_valid-y_kf_train_mean

    #Declare model
    model_kf=KalmanFilterDecoder(C=1) #There is one optional parameter (see ReadMe)

    #Fit model
    model_kf.fit(X_kf_train,y_kf_train)

    #Get predictions
    y_valid_predicted_kf=model_kf.predict(X_kf_valid,y_kf_valid)

    #Get metrics of fit (see read me for more details on the differences between metrics)
    #First I'll get the R^2
    R2_kf=get_R2(y_kf_valid,y_valid_predicted_kf)
    print('R2:',R2_kf[0:2]) #I'm just printing the R^2's of the 1st and 2nd entries that correspond to the positions
    #Next I'll get the rho^2 (the pearson correlation squared)

In [ ]:
# Only using velocity 
## Format Kinematics Data (outputs) ##
# For the Kalman filter, we use the position, velocity, and acceleration as outputs.
# Ultimately, we are only concerned with the goodness of fit of velocity, but using them all as covariates helps performance.

for i in range(len(frag_type)):
    # Pulling local variables into new, temp variables
    output = locals()["sortOut_"+frag_type[i]]
    
    # Creating a local variable for each decoder_output
    decoder_output = "decoder_output_" + frag_type[i]
    locals()[decoder_output] = []

    for j in range(len(output)): # Number of buckets (i.e. 16 or 8)
        nFrags = output[j][0].shape[0]
        temp2 = []
        for k in range(0, nFrags, 1): # Number of fragments #output[0][0].shape[0]
            vel_X = float(output[j][0][k])
            vel_Y = float(output[j][1][k])
            temp = [vel_X, vel_Y]
            temp2.append(np.array(temp))
        locals()[decoder_output].append(np.array(temp2))


In [ ]:
# Example Code for Concatenating Arrays from a List of Arrays

listArrays = final_y_test_AD[0][0:3]
listArrays

sampleList = [1,2]

slipList = [listArrays[i] for i in sampleList]
slip = np.concatenate(slipList, axis=0)

len(slipList)
slip.shape